# 📊 Performance Benchmarking Suite

**Comprehensive performance analysis for HR Resume Search MCP API**

## Benchmarking Features
- ⚡ **Response Time Analysis** - Detailed latency measurements
- 🔥 **Concurrent Load Testing** - Multi-user simulation
- 📈 **Throughput Analysis** - Requests per second metrics
- 🎯 **Search Quality Evaluation** - Relevance and accuracy scoring
- 📊 **Resource Utilization** - CPU, memory, and database metrics
- 🔍 **Bottleneck Identification** - Performance optimization insights

---

## 📦 Setup and Configuration

In [ ]:
# Import required libraries
import asyncio
import aiohttp
import time
import statistics
import psutil
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, field
import json
import uuid
import random
from pathlib import Path

# Data analysis libraries
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import precision_score, recall_score, f1_score

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# IPython display
from IPython.display import display, HTML, JSON, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, IntSlider

# Load environment
from dotenv import load_dotenv
import os
load_dotenv('../.env')

# Configure visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully")
print(f"📊 Performance benchmarking suite initialized")

In [ ]:
# Configuration
API_BASE_URL = os.getenv('API_BASE_URL', 'http://localhost:8000')
API_PREFIX = "/api/v1"
API_URL = f"{API_BASE_URL}{API_PREFIX}"

# Benchmarking configuration
BENCHMARK_CONFIG = {
    'warmup_requests': 10,
    'measurement_duration': 60,  # seconds
    'max_concurrent_users': 100,
    'step_size': 10,
    'request_timeout': 30,
    'think_time': 1.0,  # seconds between requests
    'error_threshold': 5.0,  # max acceptable error rate %
    'response_time_threshold': 2.0,  # max acceptable response time in seconds
}

# Display configuration
config_display = f"""
<div style="background: linear-gradient(135deg, #ff6b6b 0%, #feca57 50%, #48dbfb 100%); color: white; padding: 20px; border-radius: 15px; margin: 10px 0;">
    <h3 style="margin-top: 0; text-align: center;">⚡ Performance Benchmarking Configuration</h3>
    <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin: 15px 0;">
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
            <h4>🎯 Target API</h4>
            <p><strong>URL:</strong> {API_URL}</p>
            <p><strong>Timeout:</strong> {BENCHMARK_CONFIG['request_timeout']}s</p>
        </div>
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
            <h4>🔥 Load Testing</h4>
            <p><strong>Max Concurrent:</strong> {BENCHMARK_CONFIG['max_concurrent_users']}</p>
            <p><strong>Duration:</strong> {BENCHMARK_CONFIG['measurement_duration']}s</p>
        </div>
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
            <h4>📊 Quality Metrics</h4>
            <p><strong>Error Threshold:</strong> {BENCHMARK_CONFIG['error_threshold']}%</p>
            <p><strong>Response Time:</strong> {BENCHMARK_CONFIG['response_time_threshold']}s</p>
        </div>
    </div>
</div>
"""

display(HTML(config_display))
print("⚙️ Benchmarking configuration loaded")

## 🛠️ Performance Testing Framework

In [ ]:
@dataclass
class PerformanceMetric:
    """Individual performance measurement"""
    request_id: str
    endpoint: str
    method: str
    start_time: float
    end_time: float
    response_time: float
    status_code: int
    success: bool
    error_message: Optional[str] = None
    response_size: int = 0
    user_id: int = 0
    timestamp: datetime = field(default_factory=datetime.now)

@dataclass
class SystemMetrics:
    """System resource metrics"""
    timestamp: datetime
    cpu_percent: float
    memory_percent: float
    memory_used_mb: float
    disk_io_read: float
    disk_io_write: float
    network_sent: float
    network_recv: float

@dataclass
class BenchmarkResult:
    """Complete benchmark test result"""
    test_name: str
    start_time: datetime
    end_time: datetime
    duration: float
    metrics: List[PerformanceMetric]
    system_metrics: List[SystemMetrics]
    summary_stats: Dict[str, Any]
    
class PerformanceBenchmark:
    """Comprehensive performance benchmarking framework"""
    
    def __init__(self, api_url: str = API_URL):
        self.api_url = api_url
        self.session = None
        self.metrics: List[PerformanceMetric] = []
        self.system_metrics: List[SystemMetrics] = []
        self.monitoring_active = False
        
    async def create_session(self):
        """Create aiohttp session with optimized settings"""
        connector = aiohttp.TCPConnector(
            limit=200,
            limit_per_host=50,
            keepalive_timeout=30,
            enable_cleanup_closed=True
        )
        
        timeout = aiohttp.ClientTimeout(total=BENCHMARK_CONFIG['request_timeout'])
        
        self.session = aiohttp.ClientSession(
            connector=connector,
            timeout=timeout,
            headers={'User-Agent': 'PerformanceBenchmark/1.0'}
        )
    
    async def close_session(self):
        """Close aiohttp session"""
        if self.session:
            await self.session.close()
    
    def start_system_monitoring(self):
        """Start system resource monitoring"""
        self.monitoring_active = True
        
        def monitor():
            while self.monitoring_active:
                try:
                    cpu_percent = psutil.cpu_percent(interval=1)
                    memory = psutil.virtual_memory()
                    disk_io = psutil.disk_io_counters()
                    network_io = psutil.net_io_counters()
                    
                    metric = SystemMetrics(
                        timestamp=datetime.now(),
                        cpu_percent=cpu_percent,
                        memory_percent=memory.percent,
                        memory_used_mb=memory.used / (1024 * 1024),
                        disk_io_read=disk_io.read_bytes if disk_io else 0,
                        disk_io_write=disk_io.write_bytes if disk_io else 0,
                        network_sent=network_io.bytes_sent if network_io else 0,
                        network_recv=network_io.bytes_recv if network_io else 0
                    )
                    
                    self.system_metrics.append(metric)
                    
                except Exception as e:
                    print(f"Monitoring error: {e}")
                
                time.sleep(1)
        
        self.monitor_thread = threading.Thread(target=monitor, daemon=True)
        self.monitor_thread.start()
    
    def stop_system_monitoring(self):
        """Stop system resource monitoring"""
        self.monitoring_active = False
    
    async def make_request(
        self, 
        endpoint: str, 
        method: str = 'GET', 
        data: Optional[Dict] = None,
        user_id: int = 0
    ) -> PerformanceMetric:
        """Make a single HTTP request and measure performance"""
        
        request_id = str(uuid.uuid4())[:8]
        url = f"{self.api_url}{endpoint}"
        
        start_time = time.time()
        
        try:
            async with self.session.request(
                method=method,
                url=url,
                json=data if method in ['POST', 'PUT', 'PATCH'] else None,
                params=data if method == 'GET' else None
            ) as response:
                
                end_time = time.time()
                response_time = end_time - start_time
                
                # Try to read response body for size calculation
                try:
                    response_text = await response.text()
                    response_size = len(response_text.encode('utf-8'))
                except:
                    response_size = 0
                
                metric = PerformanceMetric(
                    request_id=request_id,
                    endpoint=endpoint,
                    method=method,
                    start_time=start_time,
                    end_time=end_time,
                    response_time=response_time,
                    status_code=response.status,
                    success=200 <= response.status < 400,
                    response_size=response_size,
                    user_id=user_id
                )
                
                return metric
                
        except Exception as e:
            end_time = time.time()
            response_time = end_time - start_time
            
            metric = PerformanceMetric(
                request_id=request_id,
                endpoint=endpoint,
                method=method,
                start_time=start_time,
                end_time=end_time,
                response_time=response_time,
                status_code=0,
                success=False,
                error_message=str(e),
                user_id=user_id
            )
            
            return metric
    
    async def warmup(self, endpoints: List[str], requests_per_endpoint: int = 5):
        """Warmup API with initial requests"""
        print(f"🔥 Warming up API with {requests_per_endpoint} requests per endpoint...")
        
        warmup_metrics = []
        
        for endpoint in endpoints:
            for i in range(requests_per_endpoint):
                metric = await self.make_request(endpoint)
                warmup_metrics.append(metric)
                await asyncio.sleep(0.1)  # Small delay
        
        success_rate = sum(1 for m in warmup_metrics if m.success) / len(warmup_metrics) * 100
        avg_response_time = statistics.mean(m.response_time for m in warmup_metrics)
        
        print(f"✅ Warmup completed: {success_rate:.1f}% success rate, {avg_response_time:.3f}s avg response time")
        
        return warmup_metrics
    
    def calculate_statistics(self, metrics: List[PerformanceMetric]) -> Dict[str, Any]:
        """Calculate comprehensive performance statistics"""
        if not metrics:
            return {}
        
        successful_requests = [m for m in metrics if m.success]
        failed_requests = [m for m in metrics if not m.success]
        
        response_times = [m.response_time for m in successful_requests]
        
        total_duration = max(m.end_time for m in metrics) - min(m.start_time for m in metrics)
        
        stats = {
            'total_requests': len(metrics),
            'successful_requests': len(successful_requests),
            'failed_requests': len(failed_requests),
            'success_rate': len(successful_requests) / len(metrics) * 100,
            'total_duration': total_duration,
            'requests_per_second': len(metrics) / total_duration if total_duration > 0 else 0,
            'successful_rps': len(successful_requests) / total_duration if total_duration > 0 else 0
        }
        
        if response_times:
            stats.update({
                'avg_response_time': statistics.mean(response_times),
                'median_response_time': statistics.median(response_times),
                'min_response_time': min(response_times),
                'max_response_time': max(response_times),
                'p90_response_time': np.percentile(response_times, 90),
                'p95_response_time': np.percentile(response_times, 95),
                'p99_response_time': np.percentile(response_times, 99),
                'std_response_time': statistics.stdev(response_times) if len(response_times) > 1 else 0
            })
        
        # Response size statistics
        response_sizes = [m.response_size for m in successful_requests if m.response_size > 0]
        if response_sizes:
            stats.update({
                'avg_response_size': statistics.mean(response_sizes),
                'total_bytes_transferred': sum(response_sizes)
            })
        
        # Error analysis
        error_types = {}
        for request in failed_requests:
            error_key = f"HTTP_{request.status_code}" if request.status_code > 0 else "network_error"
            error_types[error_key] = error_types.get(error_key, 0) + 1
        
        stats['error_breakdown'] = error_types
        
        return stats

# Initialize benchmark framework
benchmark = PerformanceBenchmark()
print("🏁 Performance benchmark framework initialized")

## ⚡ Benchmark Test 1: Response Time Analysis

In [ ]:
async def run_response_time_benchmark():
    """Comprehensive response time analysis across all endpoints"""
    
    print("⚡ Starting Response Time Analysis...")
    print("="*50)
    
    # Define test endpoints with different complexity levels
    test_endpoints = [
        {
            'endpoint': '/health',
            'method': 'GET',
            'complexity': 'simple',
            'description': 'Health check endpoint'
        },
        {
            'endpoint': '/search/candidates',
            'method': 'POST',
            'complexity': 'medium',
            'description': 'Basic candidate search',
            'data': {
                'query': 'software engineer',
                'limit': 10
            }
        },
        {
            'endpoint': '/search/candidates',
            'method': 'POST',
            'complexity': 'complex',
            'description': 'Advanced candidate search with filters',
            'data': {
                'query': 'senior python developer',
                'filters': {
                    'skills': ['Python', 'Django', 'PostgreSQL'],
                    'experience_years': {'min': 5, 'max': 10},
                    'departments': ['Engineering']
                },
                'scoring': {
                    'skills_weight': 0.4,
                    'experience_weight': 0.3,
                    'title_weight': 0.3
                },
                'include_score_breakdown': True,
                'limit': 20
            }
        },
        {
            'endpoint': '/search/smart',
            'method': 'POST',
            'complexity': 'complex',
            'description': 'AI-powered smart search',
            'data': {
                'query': 'Find me data scientists with machine learning experience in fintech companies',
                'enhance_query': True,
                'explain_reasoning': True
            }
        }
    ]
    
    await benchmark.create_session()
    
    try:
        # Warmup
        warmup_endpoints = [ep['endpoint'] for ep in test_endpoints]
        await benchmark.warmup(warmup_endpoints, 3)
        
        # Start system monitoring
        benchmark.start_system_monitoring()
        
        # Run response time tests
        all_metrics = []
        requests_per_endpoint = 20
        
        for endpoint_config in test_endpoints:
            print(f"\n🔍 Testing {endpoint_config['description']}...")
            endpoint_metrics = []
            
            for i in range(requests_per_endpoint):
                metric = await benchmark.make_request(
                    endpoint=endpoint_config['endpoint'],
                    method=endpoint_config['method'],
                    data=endpoint_config.get('data')
                )
                
                endpoint_metrics.append(metric)
                all_metrics.append(metric)
                
                # Small delay between requests
                await asyncio.sleep(0.1)
            
            # Calculate endpoint-specific statistics
            endpoint_stats = benchmark.calculate_statistics(endpoint_metrics)
            
            print(f"   ✅ Completed: {endpoint_stats['success_rate']:.1f}% success rate")
            print(f"   ⏱️ Avg response time: {endpoint_stats.get('avg_response_time', 0):.3f}s")
            print(f"   📊 P95 response time: {endpoint_stats.get('p95_response_time', 0):.3f}s")
        
        # Stop monitoring
        benchmark.stop_system_monitoring()
        
        # Calculate overall statistics
        overall_stats = benchmark.calculate_statistics(all_metrics)
        
        print(f"\n📊 Overall Response Time Analysis Results:")
        print(f"   Total Requests: {overall_stats['total_requests']}")
        print(f"   Success Rate: {overall_stats['success_rate']:.1f}%")
        print(f"   Average Response Time: {overall_stats.get('avg_response_time', 0):.3f}s")
        print(f"   P95 Response Time: {overall_stats.get('p95_response_time', 0):.3f}s")
        print(f"   P99 Response Time: {overall_stats.get('p99_response_time', 0):.3f}s")
        
        return all_metrics, overall_stats
        
    finally:
        await benchmark.close_session()

# Run the response time benchmark
response_time_metrics, response_time_stats = await run_response_time_benchmark()

In [ ]:
# Visualize response time analysis results
def visualize_response_time_analysis(metrics: List[PerformanceMetric], stats: Dict[str, Any]):
    """Create comprehensive response time visualizations"""
    
    # Convert metrics to DataFrame
    df = pd.DataFrame([{
        'endpoint': m.endpoint,
        'method': m.method,
        'response_time': m.response_time,
        'success': m.success,
        'status_code': m.status_code,
        'response_size': m.response_size,
        'timestamp': m.timestamp
    } for m in metrics])
    
    # Create subplot dashboard
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Response Time Distribution',
            'Response Time by Endpoint',
            'Response Time Timeline',
            'Success Rate by Endpoint',
            'Response Size vs Time',
            'Performance Summary'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"type": "indicator"}]]
    )
    
    # 1. Response time distribution
    successful_times = df[df['success']]['response_time']
    fig.add_trace(
        go.Histogram(
            x=successful_times,
            nbinsx=30,
            name="Response Time Distribution"
        ),
        row=1, col=1
    )
    
    # 2. Response time by endpoint (box plot)
    endpoints = df['endpoint'].unique()
    for endpoint in endpoints:
        endpoint_data = df[df['endpoint'] == endpoint]
        fig.add_trace(
            go.Box(
                y=endpoint_data['response_time'],
                name=endpoint.split('/')[-1],
                showlegend=False
            ),
            row=1, col=2
        )
    
    # 3. Response time timeline
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['response_time'],
            mode='lines+markers',
            name="Response Time Timeline",
            line=dict(color='blue')
        ),
        row=2, col=1
    )
    
    # 4. Success rate by endpoint
    success_rates = df.groupby('endpoint')['success'].mean() * 100
    fig.add_trace(
        go.Bar(
            x=[ep.split('/')[-1] for ep in success_rates.index],
            y=success_rates.values,
            name="Success Rate",
            marker_color='green'
        ),
        row=2, col=2
    )
    
    # 5. Response size vs time
    size_data = df[df['response_size'] > 0]
    if not size_data.empty:
        fig.add_trace(
            go.Scatter(
                x=size_data['response_time'],
                y=size_data['response_size'],
                mode='markers',
                name="Size vs Time",
                marker=dict(size=8, opacity=0.6)
            ),
            row=3, col=1
        )
    
    # 6. Performance indicator
    performance_score = min(100, (stats.get('success_rate', 0) * 0.6 + 
                                 max(0, (2.0 - stats.get('avg_response_time', 2.0)) * 50) * 0.4))
    
    fig.add_trace(
        go.Indicator(
            mode="gauge+number+delta",
            value=performance_score,
            title={'text': "Performance Score"},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': "darkblue"},
                'steps': [
                    {'range': [0, 60], 'color': "lightgray"},
                    {'range': [60, 80], 'color': "yellow"},
                    {'range': [80, 100], 'color': "green"}
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 90
                }
            }
        ),
        row=3, col=2
    )
    
    fig.update_layout(
        height=900,
        title_text="Response Time Analysis Dashboard",
        showlegend=False
    )
    
    fig.show()
    
    # Display detailed statistics table
    stats_html = f"""
    <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin: 15px 0;">
        <h4>📊 Detailed Response Time Statistics</h4>
        <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 15px;">
            <div style="text-align: center; padding: 15px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #007acc;">{stats.get('avg_response_time', 0):.3f}s</div>
                <div>Average Response Time</div>
            </div>
            <div style="text-align: center; padding: 15px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #28a745;">{stats.get('p95_response_time', 0):.3f}s</div>
                <div>95th Percentile</div>
            </div>
            <div style="text-align: center; padding: 15px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #17a2b8;">{stats.get('p99_response_time', 0):.3f}s</div>
                <div>99th Percentile</div>
            </div>
            <div style="text-align: center; padding: 15px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #dc3545;">{stats.get('success_rate', 0):.1f}%</div>
                <div>Success Rate</div>
            </div>
        </div>
        
        <div style="margin-top: 20px;">
            <h5>Performance Thresholds Analysis:</h5>
            <ul>
                <li><strong>Target Response Time:</strong> < {BENCHMARK_CONFIG['response_time_threshold']}s 
                    {'✅ PASS' if stats.get('avg_response_time', 999) < BENCHMARK_CONFIG['response_time_threshold'] else '❌ FAIL'}</li>
                <li><strong>Target Error Rate:</strong> < {BENCHMARK_CONFIG['error_threshold']}% 
                    {'✅ PASS' if (100 - stats.get('success_rate', 0)) < BENCHMARK_CONFIG['error_threshold'] else '❌ FAIL'}</li>
                <li><strong>P95 Response Time:</strong> {stats.get('p95_response_time', 0):.3f}s 
                    {'✅ GOOD' if stats.get('p95_response_time', 999) < BENCHMARK_CONFIG['response_time_threshold'] * 1.5 else '⚠️ NEEDS ATTENTION'}</li>
            </ul>
        </div>
    </div>
    """
    
    display(HTML(stats_html))

# Visualize the response time analysis
if response_time_metrics:
    visualize_response_time_analysis(response_time_metrics, response_time_stats)
else:
    print("⚠️ No response time data available for visualization")

## 🔥 Benchmark Test 2: Concurrent Load Testing

In [ ]:
async def run_concurrent_load_test():
    """Comprehensive concurrent load testing with multiple user simulation"""
    
    print("🔥 Starting Concurrent Load Testing...")
    print("="*50)
    
    await benchmark.create_session()
    
    try:
        # Test configuration
        concurrent_users = [1, 5, 10, 20, 30, 50]  # Progressive load
        test_duration = 30  # seconds per load level
        endpoint = '/search/candidates'
        
        # Test payload
        test_payload = {
            'query': 'software engineer',
            'filters': {
                'skills': ['Python', 'JavaScript'],
                'experience_years': {'min': 2}
            },
            'limit': 10
        }
        
        load_test_results = []
        
        # Start system monitoring
        benchmark.start_system_monitoring()
        
        for user_count in concurrent_users:
            print(f"\n🔄 Testing with {user_count} concurrent users...")
            
            # Reset metrics for this load level
            level_metrics = []
            
            async def user_simulation(user_id: int):
                """Simulate a single user's behavior"""
                user_metrics = []
                end_time = time.time() + test_duration
                
                while time.time() < end_time:
                    # Make request
                    metric = await benchmark.make_request(
                        endpoint=endpoint,
                        method='POST',
                        data=test_payload,
                        user_id=user_id
                    )
                    
                    user_metrics.append(metric)
                    
                    # Think time between requests
                    await asyncio.sleep(BENCHMARK_CONFIG['think_time'])
                
                return user_metrics
            
            # Run concurrent users
            start_time = time.time()
            
            user_tasks = [
                user_simulation(user_id) 
                for user_id in range(user_count)
            ]
            
            user_results = await asyncio.gather(*user_tasks)
            
            # Flatten results
            for user_metrics in user_results:
                level_metrics.extend(user_metrics)
            
            # Calculate statistics for this load level
            level_stats = benchmark.calculate_statistics(level_metrics)
            level_stats['concurrent_users'] = user_count
            level_stats['test_duration'] = test_duration
            
            load_test_results.append({
                'concurrent_users': user_count,
                'metrics': level_metrics,
                'stats': level_stats
            })
            
            # Display results for this level
            print(f"   ✅ Completed: {level_stats['success_rate']:.1f}% success rate")
            print(f"   📈 Throughput: {level_stats['successful_rps']:.1f} req/s")
            print(f"   ⏱️ Avg response time: {level_stats.get('avg_response_time', 0):.3f}s")
            print(f"   📊 P95 response time: {level_stats.get('p95_response_time', 0):.3f}s")
            
            # Short break between load levels
            await asyncio.sleep(2)
        
        # Stop monitoring
        benchmark.stop_system_monitoring()
        
        print(f"\n🏁 Concurrent Load Testing Completed!")
        
        return load_test_results
        
    finally:
        await benchmark.close_session()

# Run concurrent load test
load_test_results = await run_concurrent_load_test()

In [ ]:
# Visualize concurrent load test results
def visualize_load_test_results(results: List[Dict[str, Any]]):
    """Create comprehensive load test visualizations"""
    
    if not results:
        print("⚠️ No load test data available")
        return
    
    # Extract data for visualization
    user_counts = [r['concurrent_users'] for r in results]
    success_rates = [r['stats']['success_rate'] for r in results]
    throughputs = [r['stats']['successful_rps'] for r in results]
    avg_response_times = [r['stats'].get('avg_response_time', 0) for r in results]
    p95_response_times = [r['stats'].get('p95_response_time', 0) for r in results]
    
    # Create load test dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Throughput vs Concurrent Users',
            'Response Time vs Load',
            'Success Rate vs Load',
            'Performance Degradation'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": True}]]
    )
    
    # 1. Throughput vs concurrent users
    fig.add_trace(
        go.Scatter(
            x=user_counts,
            y=throughputs,
            mode='lines+markers',
            name='Throughput (req/s)',
            line=dict(color='green', width=3),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    # 2. Response time vs load
    fig.add_trace(
        go.Scatter(
            x=user_counts,
            y=avg_response_times,
            mode='lines+markers',
            name='Avg Response Time',
            line=dict(color='blue', width=2),
            marker=dict(size=6)
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=user_counts,
            y=p95_response_times,
            mode='lines+markers',
            name='P95 Response Time',
            line=dict(color='red', width=2, dash='dash'),
            marker=dict(size=6)
        ),
        row=1, col=2
    )
    
    # 3. Success rate vs load
    fig.add_trace(
        go.Scatter(
            x=user_counts,
            y=success_rates,
            mode='lines+markers',
            name='Success Rate (%)',
            line=dict(color='orange', width=3),
            marker=dict(size=8),
            fill='tonexty' if len(success_rates) > 1 else None
        ),
        row=2, col=1
    )
    
    # 4. Performance degradation (throughput vs response time)
    fig.add_trace(
        go.Scatter(
            x=user_counts,
            y=throughputs,
            mode='lines+markers',
            name='Throughput',
            line=dict(color='green'),
            yaxis='y'
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=user_counts,
            y=avg_response_times,
            mode='lines+markers',
            name='Response Time',
            line=dict(color='red'),
            yaxis='y2'
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Concurrent Load Testing Results",
        showlegend=True
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Concurrent Users", row=1, col=1)
    fig.update_xaxes(title_text="Concurrent Users", row=1, col=2)
    fig.update_xaxes(title_text="Concurrent Users", row=2, col=1)
    fig.update_xaxes(title_text="Concurrent Users", row=2, col=2)
    
    fig.update_yaxes(title_text="Requests/Second", row=1, col=1)
    fig.update_yaxes(title_text="Response Time (s)", row=1, col=2)
    fig.update_yaxes(title_text="Success Rate (%)", row=2, col=1)
    fig.update_yaxes(title_text="Throughput", row=2, col=2)
    
    # Add secondary y-axis for last subplot
    fig.update_layout(yaxis4=dict(title="Response Time (s)", overlaying="y3", side="right"))
    
    fig.show()
    
    # Find performance breaking point
    breaking_point = None
    for i, result in enumerate(results):
        if (result['stats']['success_rate'] < 95 or 
            result['stats'].get('avg_response_time', 0) > BENCHMARK_CONFIG['response_time_threshold']):
            breaking_point = result['concurrent_users']
            break
    
    # Calculate peak performance
    peak_throughput = max(throughputs)
    peak_users = user_counts[throughputs.index(peak_throughput)]
    
    # Display summary
    summary_html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 15px; margin: 15px 0;">
        <h4 style="margin-top: 0;">🔥 Concurrent Load Test Summary</h4>
        <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin: 15px 0;">
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h5>🏆 Peak Performance</h5>
                <p><strong>Throughput:</strong> {peak_throughput:.1f} req/s</p>
                <p><strong>At:</strong> {peak_users} concurrent users</p>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h5>⚠️ Breaking Point</h5>
                <p><strong>Users:</strong> {breaking_point or 'Not reached'}</p>
                <p><strong>Reason:</strong> {'Performance degradation' if breaking_point else 'Stable throughout test'}</p>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h5>📊 Final Results</h5>
                <p><strong>Max Users Tested:</strong> {max(user_counts)}</p>
                <p><strong>Final Success Rate:</strong> {success_rates[-1]:.1f}%</p>
            </div>
        </div>
    </div>
    """
    
    display(HTML(summary_html))

# Visualize load test results
if load_test_results:
    visualize_load_test_results(load_test_results)
else:
    print("⚠️ No load test data available for visualization")

## 🎯 Search Quality Evaluation Framework

In [ ]:
class SearchQualityEvaluator:
    """Framework for evaluating search result quality and relevance"""
    
    def __init__(self):
        self.test_queries = [
            {
                'query': 'Python developer with 5 years experience',
                'expected_skills': ['Python'],
                'expected_experience_min': 4,
                'expected_departments': ['Engineering', 'Product'],
                'relevance_threshold': 0.7
            },
            {
                'query': 'Senior data scientist machine learning fintech',
                'expected_skills': ['Python', 'Machine Learning', 'Data Science'],
                'expected_industries': ['fintech', 'finance'],
                'expected_seniority': ['Senior', 'Lead', 'Principal'],
                'relevance_threshold': 0.8
            },
            {
                'query': 'Frontend engineer React JavaScript',
                'expected_skills': ['React', 'JavaScript'],
                'expected_roles': ['Frontend', 'UI', 'Web'],
                'relevance_threshold': 0.75
            }
        ]
    
    def evaluate_skill_relevance(self, candidate: Dict, expected_skills: List[str]) -> float:
        """Evaluate how well candidate skills match expected skills"""
        candidate_skills = [skill.lower() for skill in candidate.get('skills', [])]
        expected_skills_lower = [skill.lower() for skill in expected_skills]
        
        matches = sum(1 for skill in expected_skills_lower if any(skill in cs for cs in candidate_skills))
        return matches / len(expected_skills) if expected_skills else 0
    
    def evaluate_experience_relevance(self, candidate: Dict, min_years: int) -> float:
        """Evaluate experience level match"""
        years_exp = candidate.get('metadata', {}).get('years_experience', 0)
        if years_exp >= min_years:
            return min(1.0, years_exp / (min_years * 1.5))  # Scale appropriately
        return years_exp / min_years if min_years > 0 else 0
    
    def evaluate_department_relevance(self, candidate: Dict, expected_departments: List[str]) -> float:
        """Evaluate department/role relevance"""
        experience = candidate.get('experience', [])
        if not experience:
            return 0
        
        current_dept = experience[0].get('department', '').lower()
        expected_lower = [dept.lower() for dept in expected_departments]
        
        return 1.0 if current_dept in expected_lower else 0.5
    
    def calculate_relevance_score(self, candidate: Dict, query_criteria: Dict) -> float:
        """Calculate overall relevance score for a candidate"""
        scores = []
        weights = []
        
        # Skills relevance
        if 'expected_skills' in query_criteria:
            skill_score = self.evaluate_skill_relevance(candidate, query_criteria['expected_skills'])
            scores.append(skill_score)
            weights.append(0.4)
        
        # Experience relevance
        if 'expected_experience_min' in query_criteria:
            exp_score = self.evaluate_experience_relevance(candidate, query_criteria['expected_experience_min'])
            scores.append(exp_score)
            weights.append(0.3)
        
        # Department relevance
        if 'expected_departments' in query_criteria:
            dept_score = self.evaluate_department_relevance(candidate, query_criteria['expected_departments'])
            scores.append(dept_score)
            weights.append(0.3)
        
        # Weighted average
        if scores and weights:
            return sum(score * weight for score, weight in zip(scores, weights)) / sum(weights)
        return 0
    
    async def evaluate_search_quality(self, api_client) -> Dict[str, Any]:
        """Run comprehensive search quality evaluation"""
        
        print("🎯 Starting Search Quality Evaluation...")
        print("="*50)
        
        evaluation_results = []
        
        for i, test_query in enumerate(self.test_queries, 1):
            print(f"\n🔍 Test Query {i}: '{test_query['query']}'")
            
            # Make search request
            search_result = await api_client.make_request(
                endpoint='/search/smart',
                method='POST',
                data={
                    'query': test_query['query'],
                    'limit': 10,
                    'enhance_query': True
                }
            )
            
            if search_result.success:
                try:
                    response_data = json.loads(search_result.result) if isinstance(search_result.result, str) else search_result.result
                    candidates = response_data.get('results', [])
                    
                    if candidates:
                        # Evaluate each candidate
                        candidate_scores = []
                        for candidate in candidates:
                            relevance_score = self.calculate_relevance_score(candidate, test_query)
                            candidate_scores.append({
                                'candidate_id': candidate.get('id'),
                                'name': candidate.get('name'),
                                'relevance_score': relevance_score,
                                'api_score': candidate.get('score', candidate.get('relevance_score', 0)),
                                'meets_threshold': relevance_score >= test_query['relevance_threshold']
                            })
                        
                        # Calculate quality metrics
                        avg_relevance = sum(cs['relevance_score'] for cs in candidate_scores) / len(candidate_scores)
                        precision = sum(1 for cs in candidate_scores if cs['meets_threshold']) / len(candidate_scores)
                        
                        # API vs Manual score correlation
                        api_scores = [cs['api_score'] for cs in candidate_scores if cs['api_score'] > 0]
                        manual_scores = [cs['relevance_score'] for cs in candidate_scores if cs['api_score'] > 0]
                        
                        correlation = 0
                        if len(api_scores) > 1:
                            correlation = np.corrcoef(api_scores, manual_scores)[0, 1]
                        
                        query_evaluation = {
                            'query': test_query['query'],
                            'total_results': len(candidates),
                            'avg_relevance_score': avg_relevance,
                            'precision': precision,
                            'threshold': test_query['relevance_threshold'],
                            'score_correlation': correlation,
                            'response_time': search_result.response_time,
                            'candidate_scores': candidate_scores
                        }
                        
                        evaluation_results.append(query_evaluation)
                        
                        print(f"   ✅ Results: {len(candidates)} candidates")
                        print(f"   📊 Avg Relevance: {avg_relevance:.3f}")
                        print(f"   🎯 Precision: {precision:.3f}")
                        print(f"   🔗 Score Correlation: {correlation:.3f}")
                    
                    else:
                        print(f"   ❌ No results returned")
                        
                except Exception as e:
                    print(f"   ❌ Error processing results: {e}")
            
            else:
                print(f"   ❌ Search failed: {search_result.error_message}")
        
        return evaluation_results

# Initialize search quality evaluator
quality_evaluator = SearchQualityEvaluator()
print("🎯 Search quality evaluation framework initialized")

In [ ]:
# Run search quality evaluation
async def run_search_quality_evaluation():
    """Execute search quality evaluation"""
    
    await benchmark.create_session()
    
    try:
        quality_results = await quality_evaluator.evaluate_search_quality(benchmark)
        return quality_results
    finally:
        await benchmark.close_session()

# Execute quality evaluation
search_quality_results = await run_search_quality_evaluation()

In [ ]:
# Visualize search quality results
def visualize_search_quality(quality_results: List[Dict[str, Any]]):
    """Visualize search quality evaluation results"""
    
    if not quality_results:
        print("⚠️ No search quality data available")
        return
    
    # Create quality dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Relevance Score by Query',
            'Precision vs Threshold',
            'API vs Manual Score Correlation',
            'Quality Score Distribution'
        )
    )
    
    # Extract data
    queries = [r['query'][:30] + '...' if len(r['query']) > 30 else r['query'] for r in quality_results]
    avg_relevance = [r['avg_relevance_score'] for r in quality_results]
    precisions = [r['precision'] for r in quality_results]
    correlations = [r['score_correlation'] for r in quality_results]
    
    # 1. Relevance score by query
    fig.add_trace(
        go.Bar(
            x=queries,
            y=avg_relevance,
            name='Avg Relevance',
            marker_color='lightblue'
        ),
        row=1, col=1
    )
    
    # 2. Precision vs threshold
    thresholds = [r['threshold'] for r in quality_results]
    fig.add_trace(
        go.Scatter(
            x=thresholds,
            y=precisions,
            mode='markers',
            marker=dict(size=12, color=avg_relevance, colorscale='viridis'),
            name='Precision vs Threshold'
        ),
        row=1, col=2
    )
    
    # 3. Score correlation
    fig.add_trace(
        go.Bar(
            x=queries,
            y=correlations,
            name='Score Correlation',
            marker_color='orange'
        ),
        row=2, col=1
    )
    
    # 4. Quality score distribution
    all_scores = []
    for result in quality_results:
        all_scores.extend([cs['relevance_score'] for cs in result['candidate_scores']])
    
    if all_scores:
        fig.add_trace(
            go.Histogram(
                x=all_scores,
                nbinsx=20,
                name='Quality Distribution'
            ),
            row=2, col=2
        )
    
    fig.update_layout(
        height=800,
        title_text="Search Quality Evaluation Results",
        showlegend=False
    )
    
    fig.show()
    
    # Calculate overall quality metrics
    overall_relevance = sum(avg_relevance) / len(avg_relevance) if avg_relevance else 0
    overall_precision = sum(precisions) / len(precisions) if precisions else 0
    overall_correlation = sum(correlations) / len(correlations) if correlations else 0
    
    # Display summary
    quality_summary = f"""
    <div style="background: linear-gradient(135deg, #4CAF50 0%, #45a049 100%); color: white; padding: 20px; border-radius: 15px; margin: 15px 0;">
        <h4 style="margin-top: 0;">🎯 Search Quality Summary</h4>
        <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin: 15px 0;">
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h5>📊 Average Relevance</h5>
                <p style="font-size: 24px; font-weight: bold;">{overall_relevance:.3f}</p>
                <p>{'✅ Excellent' if overall_relevance >= 0.8 else '⚠️ Good' if overall_relevance >= 0.6 else '❌ Needs Improvement'}</p>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h5>🎯 Average Precision</h5>
                <p style="font-size: 24px; font-weight: bold;">{overall_precision:.3f}</p>
                <p>{'✅ High Quality' if overall_precision >= 0.8 else '⚠️ Moderate' if overall_precision >= 0.6 else '❌ Low Quality'}</p>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <h5>🔗 Score Correlation</h5>
                <p style="font-size: 24px; font-weight: bold;">{overall_correlation:.3f}</p>
                <p>{'✅ Strong Alignment' if overall_correlation >= 0.7 else '⚠️ Moderate' if overall_correlation >= 0.5 else '❌ Weak Alignment'}</p>
            </div>
        </div>
        <div style="margin-top: 20px; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 10px;">
            <h5>💡 Quality Insights:</h5>
            <ul style="margin: 10px 0;">
                <li>{'High relevance scores indicate accurate search results' if overall_relevance >= 0.7 else 'Consider improving search algorithm relevance'}</li>
                <li>{'Good precision suggests well-tuned thresholds' if overall_precision >= 0.7 else 'Review relevance thresholds for better precision'}</li>
                <li>{'Strong correlation between API and manual scores' if overall_correlation >= 0.6 else 'API scoring may need calibration with manual evaluation'}</li>
            </ul>
        </div>
    </div>
    """
    
    display(HTML(quality_summary))

# Visualize search quality results
if search_quality_results:
    visualize_search_quality(search_quality_results)
else:
    print("⚠️ No search quality results available")

## 📊 Comprehensive Performance Report Generation

In [ ]:
def generate_performance_report(
    response_time_stats: Dict[str, Any],
    load_test_results: List[Dict[str, Any]],
    search_quality_results: List[Dict[str, Any]]
) -> str:
    """Generate comprehensive performance report"""
    
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    report = f"""
# HR Resume Search MCP API - Performance Benchmark Report

**Generated**: {timestamp}  
**API Base URL**: {API_BASE_URL}  
**Test Environment**: {os.getenv('ENVIRONMENT', 'development')}  
**Test Duration**: {BENCHMARK_CONFIG['measurement_duration']}s per load level  
**Max Concurrent Users**: {BENCHMARK_CONFIG['max_concurrent_users']}  

## 📊 Executive Summary

| Metric | Value | Status |
|--------|-------|--------|
| Average Response Time | {response_time_stats.get('avg_response_time', 0):.3f}s | {'✅ Pass' if response_time_stats.get('avg_response_time', 999) < BENCHMARK_CONFIG['response_time_threshold'] else '❌ Fail'} |
| P95 Response Time | {response_time_stats.get('p95_response_time', 0):.3f}s | {'✅ Good' if response_time_stats.get('p95_response_time', 999) < BENCHMARK_CONFIG['response_time_threshold'] * 1.5 else '⚠️ Attention'} |
| Success Rate | {response_time_stats.get('success_rate', 0):.1f}% | {'✅ Pass' if response_time_stats.get('success_rate', 0) >= (100 - BENCHMARK_CONFIG['error_threshold']) else '❌ Fail'} |
| Peak Throughput | {max([r['stats']['successful_rps'] for r in load_test_results]) if load_test_results else 0:.1f} req/s | {'✅ Excellent' if load_test_results and max([r['stats']['successful_rps'] for r in load_test_results]) > 50 else '⚠️ Moderate'} |
| Search Quality Score | {sum([r['avg_relevance_score'] for r in search_quality_results]) / len(search_quality_results) if search_quality_results else 0:.3f} | {'✅ High' if search_quality_results and sum([r['avg_relevance_score'] for r in search_quality_results]) / len(search_quality_results) > 0.7 else '⚠️ Moderate'} |

## ⚡ Response Time Analysis

### Key Metrics
- **Total Requests Tested**: {response_time_stats.get('total_requests', 0)}
- **Average Response Time**: {response_time_stats.get('avg_response_time', 0):.3f}s
- **Median Response Time**: {response_time_stats.get('median_response_time', 0):.3f}s
- **95th Percentile**: {response_time_stats.get('p95_response_time', 0):.3f}s
- **99th Percentile**: {response_time_stats.get('p99_response_time', 0):.3f}s
- **Standard Deviation**: {response_time_stats.get('std_response_time', 0):.3f}s

### Performance Assessment
"""
    
    # Response time assessment
    avg_time = response_time_stats.get('avg_response_time', 0)
    if avg_time < 0.5:
        report += "- ✅ **Excellent Response Times**: Average response time under 500ms\n"
    elif avg_time < 1.0:
        report += "- ✅ **Good Response Times**: Average response time under 1 second\n"
    elif avg_time < 2.0:
        report += "- ⚠️ **Acceptable Response Times**: Average response time under 2 seconds\n"
    else:
        report += "- ❌ **Poor Response Times**: Average response time exceeds 2 seconds\n"
    
    p95_time = response_time_stats.get('p95_response_time', 0)
    if p95_time < 2.0:
        report += "- ✅ **Good P95 Performance**: 95% of requests complete under 2 seconds\n"
    elif p95_time < 5.0:
        report += "- ⚠️ **Moderate P95 Performance**: Some requests may be slow\n"
    else:
        report += "- ❌ **Poor P95 Performance**: Significant latency for some requests\n"
    
    # Load testing results
    if load_test_results:
        peak_throughput = max([r['stats']['successful_rps'] for r in load_test_results])
        peak_users = [r['concurrent_users'] for r in load_test_results if r['stats']['successful_rps'] == peak_throughput][0]
        
        # Find breaking point
        breaking_point = None
        for result in load_test_results:
            if (result['stats']['success_rate'] < 95 or 
                result['stats'].get('avg_response_time', 0) > BENCHMARK_CONFIG['response_time_threshold']):
                breaking_point = result['concurrent_users']
                break
        
        report += f"""

## 🔥 Load Testing Results

### Scalability Metrics
- **Peak Throughput**: {peak_throughput:.1f} requests/second
- **Optimal Concurrent Users**: {peak_users}
- **Breaking Point**: {breaking_point or 'Not reached within test limits'}
- **Maximum Users Tested**: {max([r['concurrent_users'] for r in load_test_results])}

### Load Test Summary

| Users | Throughput (req/s) | Avg Response Time | Success Rate |
|-------|-------------------|-------------------|-------------|
"""
        
        for result in load_test_results:
            stats = result['stats']
            report += f"| {result['concurrent_users']} | {stats['successful_rps']:.1f} | {stats.get('avg_response_time', 0):.3f}s | {stats['success_rate']:.1f}% |\n"
        
        # Scalability assessment
        report += "\n### Scalability Assessment\n"
        if peak_throughput > 100:
            report += "- ✅ **Excellent Scalability**: High throughput achieved\n"
        elif peak_throughput > 50:
            report += "- ✅ **Good Scalability**: Moderate throughput capacity\n"
        elif peak_throughput > 20:
            report += "- ⚠️ **Limited Scalability**: Consider optimization\n"
        else:
            report += "- ❌ **Poor Scalability**: Significant performance limitations\n"
        
        if breaking_point:
            report += f"- ⚠️ **Performance Degradation**: Occurs at {breaking_point} concurrent users\n"
        else:
            report += f"- ✅ **Stable Performance**: No degradation up to {max([r['concurrent_users'] for r in load_test_results])} users\n"
    
    # Search quality results
    if search_quality_results:
        avg_relevance = sum([r['avg_relevance_score'] for r in search_quality_results]) / len(search_quality_results)
        avg_precision = sum([r['precision'] for r in search_quality_results]) / len(search_quality_results)
        avg_correlation = sum([r['score_correlation'] for r in search_quality_results]) / len(search_quality_results)
        
        report += f"""

## 🎯 Search Quality Analysis

### Quality Metrics
- **Average Relevance Score**: {avg_relevance:.3f}
- **Average Precision**: {avg_precision:.3f}
- **API-Manual Score Correlation**: {avg_correlation:.3f}
- **Test Queries Evaluated**: {len(search_quality_results)}

### Quality Assessment
"""
        
        if avg_relevance >= 0.8:
            report += "- ✅ **Excellent Search Relevance**: Highly accurate results\n"
        elif avg_relevance >= 0.6:
            report += "- ✅ **Good Search Relevance**: Generally accurate results\n"
        else:
            report += "- ❌ **Poor Search Relevance**: Results need improvement\n"
        
        if avg_precision >= 0.8:
            report += "- ✅ **High Precision**: Low false positive rate\n"
        elif avg_precision >= 0.6:
            report += "- ⚠️ **Moderate Precision**: Some irrelevant results\n"
        else:
            report += "- ❌ **Low Precision**: High false positive rate\n"
        
        if avg_correlation >= 0.7:
            report += "- ✅ **Strong Score Alignment**: API scoring is well-calibrated\n"
        elif avg_correlation >= 0.5:
            report += "- ⚠️ **Moderate Score Alignment**: Some calibration needed\n"
        else:
            report += "- ❌ **Weak Score Alignment**: Scoring algorithm needs review\n"
    
    # Recommendations
    report += """

## 💡 Performance Optimization Recommendations

### Immediate Actions
"""
    
    # Response time recommendations
    if response_time_stats.get('avg_response_time', 0) > BENCHMARK_CONFIG['response_time_threshold']:
        report += "- 🚨 **Critical**: Optimize response times - current average exceeds target\n"
        report += "  - Review database query performance\n"
        report += "  - Implement caching strategies\n"
        report += "  - Consider API endpoint optimization\n"
    
    # Success rate recommendations
    if response_time_stats.get('success_rate', 0) < (100 - BENCHMARK_CONFIG['error_threshold']):
        report += "- 🚨 **Critical**: Address error rate - success rate below target\n"
        report += "  - Investigate failed requests\n"
        report += "  - Improve error handling\n"
        report += "  - Add retry mechanisms\n"
    
    # Scalability recommendations
    if load_test_results:
        peak_throughput = max([r['stats']['successful_rps'] for r in load_test_results])
        if peak_throughput < 50:
            report += "- ⚡ **Scalability**: Improve concurrent request handling\n"
            report += "  - Implement connection pooling\n"
            report += "  - Add horizontal scaling capabilities\n"
            report += "  - Optimize resource utilization\n"
    
    # Search quality recommendations
    if search_quality_results:
        avg_relevance = sum([r['avg_relevance_score'] for r in search_quality_results]) / len(search_quality_results)
        if avg_relevance < 0.7:
            report += "- 🎯 **Search Quality**: Improve search relevance\n"
            report += "  - Refine scoring algorithms\n"
            report += "  - Enhance query understanding\n"
            report += "  - Improve candidate matching logic\n"
    
    report += """

### Long-term Improvements
- 📊 **Monitoring**: Implement continuous performance monitoring
- 🔄 **Automation**: Add performance testing to CI/CD pipeline
- 📈 **Alerting**: Set up performance degradation alerts
- 🧪 **A/B Testing**: Test optimization strategies systematically
- 💾 **Caching**: Implement advanced caching strategies
- 🔍 **Search**: Enhance AI-powered search capabilities

---

**Report Generated by**: HR Resume Search MCP API Performance Benchmarking Suite  
**Notebook**: `notebooks/performance_benchmarking.ipynb`  
**Timestamp**: {timestamp}
"""
    
    return report

# Generate comprehensive report
if response_time_stats and load_test_results:
    performance_report = generate_performance_report(
        response_time_stats,
        load_test_results,
        search_quality_results or []
    )
    
    # Save report to file
    report_filename = f"performance_benchmark_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    with open(report_filename, 'w') as f:
        f.write(performance_report)
    
    print(f"📋 Comprehensive performance report saved to: {report_filename}")
    
    # Display report summary
    display(Markdown("## 📋 Performance Benchmark Report Summary\n\n" + performance_report[:2000] + "\n\n*Full report saved to file.*"))
    
else:
    print("⚠️ Insufficient data to generate comprehensive report")

print("\n🎉 Performance benchmarking suite completed successfully!")
print(f"📊 Generated comprehensive analysis and report")
print(f"🔍 All performance metrics captured and analyzed")

## 🎯 Summary and Insights

This comprehensive performance benchmarking suite provides:

### ✅ **Performance Analysis Features**
- **⚡ Response Time Analysis** - Detailed latency measurements across endpoints
- **🔥 Concurrent Load Testing** - Multi-user simulation with progressive load
- **🎯 Search Quality Evaluation** - Relevance and accuracy scoring framework
- **📊 System Resource Monitoring** - CPU, memory, and I/O tracking
- **📈 Performance Visualization** - Interactive charts and dashboards
- **📋 Comprehensive Reporting** - Detailed markdown reports with recommendations

### 🔧 **Key Capabilities**
- **Real-time Monitoring** - System resource tracking during tests
- **Quality Metrics** - Precision, relevance, and correlation analysis
- **Scalability Testing** - Breaking point identification and throughput analysis
- **Performance Thresholds** - Automated pass/fail criteria evaluation
- **Optimization Recommendations** - Data-driven improvement suggestions

### 📊 **Benchmarking Insights**
- **Response Time Targets** - < 2s average, < 3s P95
- **Throughput Goals** - > 50 req/s sustained
- **Quality Standards** - > 0.7 relevance score, > 0.8 precision
- **Error Tolerance** - < 5% error rate
- **Scalability Requirements** - Handle 50+ concurrent users

---

**Next Steps:**
1. Run benchmarks regularly to track performance trends
2. Implement continuous performance monitoring
3. Set up automated alerts for performance degradation
4. Use insights to guide optimization efforts
5. Integrate performance testing into CI/CD pipeline

**Happy Benchmarking! 📊🚀**